In [1]:
import os

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

from lib.dataset.cataracts_dataset import CATARACTSDataset
from lib.utils.misc import label_vectors_to_names

In [2]:
DATA_PATH = '/media/yannik/samsung_data_ssd/data/CATARACTS-videos-processed/'
TARGET_PATH = "results/real_qual_samples/"
DEV = 'cuda'
os.makedirs(TARGET_PATH, exist_ok=True)
BATCH_SIZE = 1
STEPS = 1  # 30000//BATCH_SIZE
TARGET_SHAPE = (3, 270, 480)
print(f"Avail. GPUs: ", torch.cuda.device_count())

Avail. GPUs:  1


In [7]:
test_ds = CATARACTSDataset(
    root=DATA_PATH,
    resize_shape=TARGET_SHAPE[1:],
    normalize=None,
    mode='val',  # 'test' / 'train
    frame_step=3,
    sample_img=True
)
# TODO: Weighted sampling / sampling from p(toolset|phase)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=8,
                     drop_last=True, shuffle=True, pin_memory=False)
print(f"{len(test_ds)} test samples")

18017 test samples


## Saving real samples for qual. comparison

In [9]:
for i, (img, _, file_name, phase_label, tool_label) in enumerate(tqdm(test_dl)):

    phase, toolset = label_vectors_to_names(test_ds, phase_label[0], tool_label[0])
    phase = phase.replace("/", " ")
    toolset = [tool.replace("/", " ") for tool in toolset]

    img = (img * 255.).type(torch.uint8).squeeze(0)
    img = img.permute(1, 2, 0).cpu().numpy()

    im = Image.fromarray(img)
    im.save(os.path.join(TARGET_PATH + f"{phase}_{toolset}_sample{i}.png"))

100%|██████████| 18017/18017 [09:58<00:00, 30.09it/s]
